In [14]:
# Import helper functions
def split_dir_to_train_test_val(directory = "data/",
                            train_size = 0.7,
                            test_size = 0.2,
                            val_size = 0.1):
  """
  Creates 3 folders for Train, Test and Validation data
  """
  import os
  import random
  import shutil

  # Set random seed
  rng = random.Random(42)

  for root, folders, files in os.walk(directory):
    for folder in folders:
      # Create list of the files
      list_of_files = []
      for file_name in os.listdir(root+folder+"/"):
        list_of_files.append(file_name)
      
      #  Shuffle the list
      rng.shuffle(list_of_files)

      # Create lists of files
      train_files = list_of_files[:int(len(list_of_files)*train_size)]
      test_files = list_of_files[int(len(list_of_files)*train_size) : int(len(list_of_files)*(train_size+test_size))]
      val_files = list_of_files[int(len(list_of_files)*(train_size+test_size)):]

      # Create folders and files for train data
      for one_file in train_files:
      
        # Copy  files
        dest_dir = "files/train/"+folder+"/"
        os.makedirs(dest_dir, exist_ok=True)

        shutil.copy2(src=(root+folder+"/"+one_file),
                    dst=(dest_dir+one_file))
      print(f"Folder {folder}. Train data copied. {len(train_files)} files")

      # Create folders and files for test data
      for one_file in test_files:      
        # Copy  files
        dest_dir = "files/test/"+folder+"/"
        os.makedirs(dest_dir, exist_ok=True)

        shutil.copy2(src=(root+folder+"/"+one_file),
                    dst=(dest_dir+one_file))
      print(f"Folder {folder}. Test data copied. {len(test_files)} files")

      # Create folders and files for validation data
      for one_file in val_files:
      
        # Copy  files
        dest_dir = "files/validation/"+folder+"/"
        os.makedirs(dest_dir, exist_ok=True)

        shutil.copy2(src=(root+folder+"/"+one_file),
                    dst=(dest_dir+one_file))
      print(f"Folder {folder}. Validation data copied. {len(val_files)} files")
      
     


def get_class_names_from_folder(directory):
  """
  Get the classnames from train folder for example
  """
  import pathlib
  import numpy as np
  data_dir = pathlib.Path(directory)
  class_names = np.array(sorted([item.name for item in data_dir.glob("*")])) # Created a list of class names 
  return class_names
  print(class_names)

In [36]:
# Split images dir to train, test and validation
split_dir_to_train_test_val(directory="data/",
                            train_size=0.7,
                            test_size=0.2,
                            val_size=0.1)

Folder kulfi. Train data copied. 93 files
Folder kulfi. Test data copied. 27 files
Folder kulfi. Validation data copied. 14 files
Folder jalebi. Train data copied. 135 files
Folder jalebi. Test data copied. 39 files
Folder jalebi. Validation data copied. 20 files
Folder burger. Train data copied. 163 files
Folder burger. Test data copied. 46 files
Folder burger. Validation data copied. 24 files
Folder pizza. Train data copied. 128 files
Folder pizza. Test data copied. 37 files
Folder pizza. Validation data copied. 19 files
Folder kadai_paneer. Train data copied. 159 files
Folder kadai_paneer. Test data copied. 46 files
Folder kadai_paneer. Validation data copied. 23 files
Folder pakode. Train data copied. 125 files
Folder pakode. Test data copied. 36 files
Folder pakode. Validation data copied. 18 files
Folder butter_naan. Train data copied. 151 files
Folder butter_naan. Test data copied. 44 files
Folder butter_naan. Validation data copied. 22 files
Folder pav_bhaji. Train data copied.

In [16]:
# Using helper function get class names
class_names = get_class_names_from_folder(directory="files/train/")
class_names

array(['burger', 'butter_naan', 'chai', 'chapati', 'chole_bhature',
       'dal_makhani', 'dhokla', 'fried_rice', 'idli', 'jalebi',
       'kaathi_rolls', 'kadai_paneer', 'kulfi', 'masala_dosa', 'momos',
       'paani_puri', 'pakode', 'pav_bhaji', 'pizza', 'samosa'],
      dtype='<U13')

In [41]:
import tensorflow as tf

train_path = "files/train/"
test_path = "files/test/"
validation_path = "files/validation/"

img_height = 180
img_width = 180

data_train = tf.keras.utils.image_dataset_from_directory(
    train_path,
    shuffle=True,
    image_size=(img_height, img_width),
    batch_size=32)

data_categs = data_train.class_names

data_test = tf.keras.utils.image_dataset_from_directory(
    test_path,
    shuffle=True,
    image_size=(img_height, img_width),
    batch_size=32)

data_val = tf.keras.utils.image_dataset_from_directory(
    validation_path,
    shuffle=True,
    image_size=(img_height, img_width),
    batch_size=32)




Found 2786 files belonging to 20 classes.
Found 799 files belonging to 20 classes.
Found 411 files belonging to 20 classes.


In [42]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

model = Sequential([
    layers.Rescaling(1./255),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(units=len(data_categs)),
])

In [44]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [45]:
history = model.fit(
    data_train,
    validation_data=data_val,
    epochs=25,
)

Epoch 1/25
88/88 ━━━━━━━━━━━━━━━━━━━━ 31s 329ms/step - accuracy: 0.0896 - loss: 2.9375 - val_accuracy: 0.1922 - val_loss: 2.6194
Epoch 2/25
88/88 ━━━━━━━━━━━━━━━━━━━━ 37s 290ms/step - accuracy: 0.2646 - loss: 2.3603 - val_accuracy: 0.2822 - val_loss: 2.3728
Epoch 3/25
88/88 ━━━━━━━━━━━━━━━━━━━━ 40s 281ms/step - accuracy: 0.4503 - loss: 1.8293 - val_accuracy: 0.3431 - val_loss: 2.4000
Epoch 4/25
88/88 ━━━━━━━━━━━━━━━━━━━━ 25s 287ms/step - accuracy: 0.6751 - loss: 1.0728 - val_accuracy: 0.2676 - val_loss: 2.8088
Epoch 5/25
88/88 ━━━━━━━━━━━━━━━━━━━━ 24s 275ms/step - accuracy: 0.8119 - loss: 0.6638 - val_accuracy: 0.3041 - val_loss: 3.3286
Epoch 6/25
88/88 ━━━━━━━━━━━━━━━━━━━━ 25s 284ms/step - accuracy: 0.9243 - loss: 0.2822 - val_accuracy: 0.3431 - val_loss: 4.1312
Epoch 7/25
88/88 ━━━━━━━━━━━━━━━━━━━━ 25s 279ms/step - accuracy: 0.9699 - loss: 0.1165 - val_accuracy: 0.3285 - val_loss: 4.1244
Epoch 8/25
88/88 ━━━━━━━━━━━━━━━━━━━━ 25s 286ms/step - accuracy: 0.9781 - loss: 0.0805 - val_accu

In [51]:
model.save('ind.keras')

In [52]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('ind.tflite', 'wb') as f:
  f.write(tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmpswg61num/assets


INFO:tensorflow:Assets written to: /tmp/tmpswg61num/assets


Saved artifact at '/tmp/tmpswg61num'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name='keras_tensor_39')
Output Type:
  TensorSpec(shape=(None, 20), dtype=tf.float32, name=None)
Captures:
  132121381266896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132121381259792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132121381263440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132121381268240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132121381265744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132121381268624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132121381265936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132121381269200: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1725644326.463802   30021 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1725644326.464339   30021 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-09-06 23:08:46.470670: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpswg61num
2024-09-06 23:08:46.471276: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-09-06 23:08:46.471296: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpswg61num
2024-09-06 23:08:46.478152: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-09-06 23:08:46.479064: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-09-06 23:08:46.534012: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpswg61num
2024-09-06 23:08:46.544377: I tensorflow/cc/saved_model/loader.cc

In [53]:
train_data.class_indices

{'burger': 0,
 'butter_naan': 1,
 'chai': 2,
 'chapati': 3,
 'chole_bhature': 4,
 'dal_makhani': 5,
 'dhokla': 6,
 'fried_rice': 7,
 'idli': 8,
 'jalebi': 9,
 'kaathi_rolls': 10,
 'kadai_paneer': 11,
 'kulfi': 12,
 'masala_dosa': 13,
 'momos': 14,
 'paani_puri': 15,
 'pakode': 16,
 'pav_bhaji': 17,
 'pizza': 18,
 'samosa': 19}